In [ ]:
import hvplot.polars  # noqa
import hydra
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
import rootutils
import torch
import torch.nn.functional as F

rootutils.setup_root("../", pythonpath=True)

In [ ]:
# setting up paths
from pathlib import Path

from dotenv import dotenv_values

paths = dotenv_values()

data_path = Path(paths["DATA_DIR"])
data_path

In [ ]:
df = pl.read_parquet(data_path / "gene_embeddings/PCA-train_expression.parquet")
df

In [ ]:
genes = df["gene_name"]
dataset = df.select(cs.numeric()).to_torch("dataset")
dataset

In [ ]:
dataloader = torch.utils.data.DataLoader(
    dataset=dataset, num_workers=1, batch_size=32, shuffle=False
)
batch = dataloader.__iter__().__next__()[0]
batch

In [ ]:
batch[0].shape

In [ ]:
pred_norm = F.normalize(batch, dim=-1)
gene_norm = F.normalize(batch, dim=-1)
pred_norm

In [ ]:
cos_sim = torch.matmul(pred_norm, pred_norm.T)
cos_sim

In [ ]:
mask = ~torch.eye(32, dtype=bool)
mask

In [ ]:
cos_sim_off_diag = cos_sim[mask].view(32, 31)

In [ ]:
cos_sim_off_diag[1, :]

In [ ]:
weights = torch.pow(cos_sim_off_diag, 2)
weights[1, :]

In [ ]:
exp_cos_sim = torch.exp(cos_sim_off_diag)
exp_cos_sim

In [ ]:
weighted_pos = (weights * exp_cos_sim).sum(dim=1)
weighted_pos

In [ ]:
total_sim = exp_cos_sim.sum(dim=1)
total_sim

In [ ]:
weighted_pos / (total_sim + 1e-8)